In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('final_dataset.csv')
df.head()

,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,year,month,cases
0,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,5,87,6,14,15,14,16,0,59,0.0,1018,24,10,189,10,2016,3,2
1,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,5,87,6,14,15,14,16,0,59,0.0,1018,24,10,189,10,2016,3,0
2,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,5,87,6,14,15,14,16,0,59,0.0,1018,24,10,189,10,2016,3,1
3,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,5,87,6,14,15,14,16,0,59,0.0,1018,24,10,189,10,2016,3,0
4,12625.800781,-34.603684,-58.381559,22,10,0.0,11.5,5,76,10,14,16,14,27,34,72,0.0,1025,22,10,99,18,2016,4,0


In [ ]:
df.shape

(174103, 25)

In [ ]:
# Remove duplicate rows
df = df.drop_duplicates(keep='first', inplace=False)

In [ ]:
df.shape

(103045, 25)

#Scaling

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# feature_scale=[feature for feature in df.columns if feature not in ['cases']]

# scaler=MinMaxScaler()
# scaler.fit(df[feature_scale])

# data = pd.concat([df[['cases']].reset_index(drop=True),
#                     pd.DataFrame(scaler.transform(df[feature_scale]), columns=feature_scale)],axis=1)

# data.head()

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso

In [ ]:
X = df.drop('cases', axis=1)
y = df['cases']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])

In [ ]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,10,0.1)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )

In [ ]:
search.fit(X_train,y_train)

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV] model__alpha=0.1 ................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .............. model__alpha=0.1, score=-494723.812, total=   0.8s
[CV] model__alpha=0.1 ................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV] .............. model__alpha=0.1, score=-565375.423, total=   0.8s
[CV] model__alpha=0.1 ................................................


...

[CV] .............. model__alpha=0.1, score=-236156.322, total=   0.8s
[CV] model__alpha=0.1 ................................................


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 95207729.68454742, tolerance: 3074978.319875654
  positive)


[CV] .............. model__alpha=0.1, score=-429004.829, total=   1.5s
[CV] model__alpha=0.1 ................................................
[CV] .............. model__alpha=0.1, score=-913740.893, total=   0.7s
[CV] model__alpha=0.2 ................................................
[CV] .............. model__alpha=0.2, score=-494691.058, total=   0.7s
[CV] model__alpha=0.2 ................................................
[CV] .............. model__alpha=0.2, score=-565453.189, total=   0.7s
[CV] model__alpha=0.2 ................................................
[CV] .............. model__alpha=0.2, score=-236122.090, total=   0.5s
[CV] model__alpha=0.2 ................................................
[CV] .............. model__alpha=0.2, score=-428989.546, total=   0.7s
[CV] model__alpha=0.2 ................................................
[CV] .............. model__alpha=0.2, score=-913743.834, total=   0.7s
[CV] model__alpha=0.30000000000000004 ................................
[CV]  

[Parallel(n_jobs=1)]: Done 495 out of 495 | elapsed:  1.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('model',
                                        Lasso(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=1000,
                                              normalize=False, positive=False,
                                              precompute=False,
                                              random_state=None,
                                              selection='cyclic', tol=0.0001,
                                              warm_start=False))],
                                verbose=False),
             iid='deprec...
       2.7, 2.8, 2.9, 3. , 3.1,

In [ ]:
search.best_params_

{'model__alpha': 0.2}

In [ ]:
coefficients = search.best_estimator_.named_steps['model'].coef_

In [ ]:
importance = np.abs(coefficients)

In [ ]:
importance

array([  9.98884354,  20.61289956,  57.97124885,   0.        ,
        14.21797841,   1.1996766 ,  19.2291611 ,  12.55088909,
         1.04060297,   7.89568768,  86.42314299,   0.        ,
       128.99530134,  43.71961936,   6.60225208,  19.43100994,
         1.07408641,  21.36223768,   8.8267877 ,   2.67336852,
        12.8910683 ,  72.72265355,   3.40463408,  11.16045446])

In [ ]:
np.array(X.columns)[importance > 0]

array(['density_per_km', 'latitude', 'longitude', 'mintempC',
       'totalSnow_cm', 'sunHour', 'uvIndex', 'moon_illumination',
       'DewPointC', 'FeelsLikeC', 'WindChillC', 'WindGustKmph',
       'cloudcover', 'humidity', 'precipMM', 'pressure', 'tempC',
       'visibility', 'winddirDegree', 'windspeedKmph', 'year', 'month'],
      dtype=object)

In [ ]:
np.array(X.columns)[importance == 0]

array(['maxtempC', 'HeatIndexC'], dtype=object)